In [ ]:
import pandas as pd

from constants.paths import INTERMEDIATE_PC_PRICE_DIR, INTERMEDIATE_PHENOL_ACETONE_DIR
from src.data_pipelines.uni_intermediate_to_processed import build_univariate_dataset
import src.utils.feature_engineering as fe_utils

# Analysing intermediate data files

In [ ]:
pc_price_eu = pd.read_csv(INTERMEDIATE_PC_PRICE_DIR / "intermediate_pc_price_eu.csv")
pc_price_asia = pd.read_csv(
    INTERMEDIATE_PC_PRICE_DIR / "intermediate_pc_price_asia.csv"
)
bpa_capacity_loss = pd.read_csv(
    INTERMEDIATE_PHENOL_ACETONE_DIR / "intermediate_bpa_capacity_loss.csv"
)

# Feature Engineering approach

There are 2 approaches to feature engineering depending on the modeling technique we want to use:
1. PC type specific models: Use a wide format dataset (1 column per PC type).
2. Global multivariate model: Use a long format dataset (1 row per PC type per country per month) with all features included.

## 1 - PC type specific models 

In this approach, we will create a wide format dataset where each PC type has its own column. Features will be engineered specifically for each PC type. We first create a large dataset with all possible features for each PC type, and select a subset of that dataset at modeling time.

The base dataset is created using the `build_univariate_dataset` function in `src/data_pipelines/uni_intermediate_to_processed.py`. This function simply concatenates all datasets (PC prices and exogenous variables) into a wide format dataframe.

In [ ]:
fe_utils.create_wide_format()

The `asia_pc_gf_best_price` column is weird. Possible that values within the column don't have the same unit. Comes from data, nothing we can really do about it...

This is the base for the pipeline that creates the final processed dataset used for modeling in `src/data_pipelines/uni_intermediate_to_processed.py`. The final dataset is built by adding features to the base dataset using functions from `src/utils/feature_engineering.py`.
The feature engineering steps are as follows:
- Calendar features (month, quarter, year) with cyclical encoding for month and quarter.
- Lag features for PC prices.
- Rolling window statistics for PC prices.
- Rate of change features for PC prices.
- Lag features for exogenous variables.
- Rolling window statistics for exogenous variables.

In [ ]:
build_univariate_dataset(horizon=3)

### Feature engineering

Once the base wide format dataset is created, we can engineer potential features (which will be filtered later). See the file `src/utils/feature_engineering.py` for the implementation of the feature engineering functions.

## 2 - Global multivariate model

In this approach , we will create a long format dataset where each row corresponds to a specific PC type in a specific region at a specific month.

```
| date       | region | pc_type | price | ... features ... |
|------------|--------|---------|-------|------------------|
| 2020-01-01 | europe | crystal | 2.5   | ...              |
| 2020-01-01 | europe | gf10    | 2.8   | ...              |
| 2020-01-01 | asia   | gp      | 2.3   | ...              |
```

In [ ]:
fe_utils.create_long_format()

# TODO - Select features for modelling

- Correlation analysis
- ADF testing (stationarity of features)
- Granger causality tests (past values of features helping predict target)
- Time-lagged mutual information
- Independence testing

# Analysis Summary

## What the Expert Found (Critical Issues with Past Approaches):

Major Problems:
1. Look-ahead bias - Both groups likely selected features on full datasets, inflating performance
2. Single-horizon selection - Used same features for 3-month and 9-month forecasts (suboptimal)
3. Exogenous availability problem - Didn't properly account for unavailable future features
4. Lag feature explosion - Created 500+ correlated features, risking overfitting
5. PCA misuse - Group 2 used PCA on tree models (loses interpretability, minimal benefit)
6. Granger causality overuse - Assumes linearity, sensitive to lag choice

## What's Good About Their Approaches:

Group 1 Strengths:
- Rigorous statistical testing (Granger + Mutual Information)
- Feature independence testing
- Lean final selection (5 features)

Group 2 Strengths:
- Multi-method ensemble approach
- Domain-specific processing (separate virgin/recycled models)
- SHAP for explainability

## What's Missing:

1. Horizon-specific feature sets - Should select different features for 3, 6, 9-month forecasts
2. Proper temporal validation - Feature selection should happen within TimeSeriesSplit folds
3. Smart lag engineering - Rolling aggregates and rate-of-change instead of raw lags
4. Availability constraints - Must ensure lag ≥ forecast horizon
5. Domain features - Price spreads, ratios, shutdown intensity metrics

# Recommended Approach for Your Project:

## Phase 1: Feature Engineering (Better than just lags)

Instead of `lag_1`, `lag_2`, `lag_3`, ..., `lag_24`:

- Rolling aggregates (smoother): `pc_price_ma3`, `pc_price_ma6`, `pc_price_ma12`

- Rate of change (momentum): `pc_price_roc3` = `pc_price.pct_change(3)`

- Domain features: `asia_europe_spread` = `pc_asia` - `pc_eu` & `capacity_loss_6m` = `bpa_capacity.rolling(6).sum()`
- Temporal (seasonality): `month_sin`, `month_cos`, `quarter`

## Phase 2: Horizon-Specific Selection

- 3-month forecast: Use lags ≥3, short-term indicators (`ma3`, `roc1`)
- 6-month forecast: Use lags ≥6, medium-term trends (`ma6`, `roc3`)
- 9-month forecast: Use lags ≥9, long-term fundamentals (`ma12`, `seasonal`)

## Phase 3: Selection Methods (Simplified Pipeline)

1. Variance threshold (remove constants)
2. Correlation with target (|r| > 0.3)
3. Random Forest importance (top 50%)
4. Remove multicollinearity (correlation > 0.9)
5. Optional: Granger causality (for interpretation only)

## Phase 4: Validation Strategy

- Use TimeSeriesSplit with 5 folds
- Select features that appear in ≥60% of folds (robustness)
- Never select on full dataset

# Key Recommendations:

✅ DO:

- Create horizon-specific datasets (`features_3m.csv`, `features_6m.csv`, `features_9m.csv`)
- Use rolling aggregates instead of raw lags
- Ensure lag ≥ forecast horizon
- Select features within CV folds (avoid leakage)
- Track experiments in MLflow
- Use DVC pipeline for reproducibility

❌ DON'T:

- Create hundreds of lag features
- Use PCA for XGBoost/Random Forest
- Select same features for all horizons
- Include features unavailable at prediction time
- Rely solely on Granger causality

Priority Actions:

Week 1:
1. Create src/data_pipelines/intermediate_to_processed.py
2. Generate rolling aggregates, rate-of-change, domain features
3. Create horizon-specific datasets

Week 2:
4. Implement feature selection with TimeSeriesSplit
5. Run for h=3, 6, 9 separately
6. Log to MLflow

Expected Results:
- 3-month: ~15-25 features
- 6-month: ~10-20 features
- 9-month: ~8-15 features
- MAPE improvement: ~30-50% better than naive baseline

------

How Fold-Based Feature Selection Works

Step-by-Step Mechanism:

# Conceptual flow:
For each of 5 CV folds:
- 1. Split data: Train (expanding) → Test
- 2. Run feature selection on TRAIN only
- 3. Record which features were selected

After all folds:
- 4. Count how many times each feature was selected
- 5. Keep features that appeared in ≥ threshold × folds

Your Concrete Example Analyzed:

Given your example where each fold selected 5 features:
- Fold 1: [A, B, C, D, E]
- Fold 2: [A, B, D, F, G]
- Fold 3: [A, C, D, F, H]
- Fold 4: [B, C, D, E, F]
- Fold 5: [A, D, F, G, I]

Feature counts:
- D: 5/5 folds (100%) → Must include (stable across all time periods)
- A, F: 4/5 folds (80%) → Very strong candidates
- B, C: 3/5 folds (60%) → Borderline at 60% threshold
- E, G: 2/5 folds (40%) → Reject at 60% threshold
- H, I: 1/5 fold (20%) → Definitely reject

At 60% threshold (3/5 folds): Select [A, B, C, D, F] (5 features)
At 80% threshold (4/5 folds): Select [A, D, F] (3 features)

Why This Is Better Than Selecting on Full Training Set:

Problem it solves:
1. Spurious correlations - If feature X is only useful in 2019-2020 but useless in 2021-2023, full-set selection might keep it. Fold-based
approach reveals it's unstable.
2. Overfitting to specific time periods - A feature that looks great on full training data might only work in one market regime.
3. Data leakage detection - If a feature is selected in early folds but not later folds, it might be "looking ahead" somehow.

Concrete example:
- Feature: covid_impact (dummy for 2020-2021)
- Full-set selection: High importance (captures 2020 price drop)
- Fold-based: Only selected in folds that include 2020 → Low consistency → Rejected
- Result: More generalizable model

Threshold Choice:

| Threshold  | Meaning                             | When to Use                                          |
|------------|-------------------------------------|------------------------------------------------------|
| 100% (5/5) | Feature must be useful in ALL folds | Very conservative, very few features, might underfit |
| 80% (4/5)  | Strong consensus                    | Good for production models, high confidence          |
| 60% (3/5)  | Majority vote                       | Default recommendation - balanced                    |
| 40% (2/5)  | Lenient                             | Small datasets, exploratory phase                    |
| 20% (1/5)  | Very lenient                        | Too permissive, defeats the purpose                  |

For your PC price case (150 observations):
- Recommended: 40% (2/5 folds)
- Reason: Small dataset means each fold has ~30 test samples, higher variance in selection
- Alternative: Use 3-fold CV with 60% threshold (2/3 folds) → larger folds, same logic

Key Adjustments for Your Small Dataset:

The expert recommended several modifications:

1. Lower threshold to 40% (2/5 folds) instead of 60%
- Reason: With only 150 observations, feature selection is noisier
2. Add 3-month embargo between train/test
tscv = TimeSeriesSplit(n_splits=5, gap=3)  # 3-month gap
- Prevents label leakage from autocorrelation
3. Consider 3-fold CV instead of 5-fold
- Larger folds → more stable selection
- 60% threshold = 2/3 folds (same as 40% with 5-fold)
4. Use feature groups to ensure diversity
FEATURE_GROUPS = {
    'autoregressive': ['pc_price_lag3', 'pc_price_lag6', ...],
    'commodities': ['oil_price_lag6', 'gas_price_lag6', ...],
    'supply': ['capacity_loss_6m', 'shutdown_count_6m', ...],
    'temporal': ['month_sin', 'month_cos', 'quarter']
}
# Ensure at least 1 feature from each group is selected

Implementation Approach:

The expert provided code showing you should:

1. Test multiple thresholds as a hyperparameter:
```python
for threshold in [0.2, 0.4, 0.6, 0.8]:
    features = select_with_threshold(threshold)
    model = train_model(features)
    performance = evaluate(model)
    mlflow.log_metrics({"rmse": performance, "threshold": threshold})
```
2. Track everything in MLflow:
- Consensus threshold (e.g., 0.4)
- Feature counts per fold
- Importance scores averaged across folds
- Final feature list
3. Validate the threshold choice:
- Plot: threshold (x-axis) vs. RMSE (y-axis)
- Look for "elbow" where performance plateaus
- Choose threshold that balances features vs. performance

Bottom Line:

For your PC price forecasting project with 150 monthly observations:

Recommended Configuration:
CV_CONFIG = {
    'n_splits': 5,           # Standard
    'gap': 3,                # 3-month embargo
    'threshold': 0.4,        # 2/5 folds (lenient for small data)
    'max_train_size': None   # Use all available history
}

Expected outcome:
- ~10-15 features selected at 40% threshold
- More robust than single-train selection
- Computational cost: 5x (acceptable for your dataset size)
- Interpretability: Can explain "feature X was useful in 4/5 time periods"

----

Use a simple, fast model for feature selection, then use those features with your actual modeling approaches (XGBoost,
Prophet, etc.).

Two-Stage Process:

# STAGE 1: Feature Selection (uses simple model)
Goal: Reduce from 100+ features to ~15-20 robust features

```python
selected_features = select_features_with_cv(
    X_train,
    y_train,
    selector_model=RandomForestRegressor(  # <-- Simple model for selection
        n_estimators=100,
        max_depth=10,
        random_state=42
    )
)
```

# STAGE 2: Actual Modeling (uses selected features)
Goal: Find best model for prediction

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Try different models with the SAME feature set:
```python
models = {
    'xgboost': XGBRegressor(...),
    'random_forest': RandomForestRegressor(...),
    'prophet': Prophet(...),
    'sarima': SARIMAX(...)
}

for name, model in models.items():
    model.fit(X_train_selected, y_train)
    predictions = model.predict(X_test_selected)
    # Compare performance...
```

Why This Works:

1. Feature selection is model-agnostic (mostly)
- Features that help Random Forest usually help XGBoost too
- Both are tree-based and capture similar relationships
- Good features for ML models often good for time-series models too
2. Selection model should be:
- Fast (you're running 5-fold CV)
- Robust (not too prone to overfitting)
- Similar to final models (if possible)

Recommended Selection Models:

| Your Final Models                | Use This For Selection                                | Why                                         |
|----------------------------------|-------------------------------------------------------|---------------------------------------------|
| XGBoost, CatBoost, Random Forest | RandomForestRegressor(n_estimators=100, max_depth=10) | Fast, similar algorithm family, robust      |
| Linear models (Ridge, Lasso)     | LassoCV or correlation-based                          | Matches model assumptions                   |
| Prophet, SARIMA                  | Correlation + domain knowledge                        | Time-series models use features differently |
| Mixed (trying everything)        | RandomForestRegressor                                 | Most versatile, works for all               |

For Your PC Price Project:

Recommended approach:

## Stage 1: Feature Selection
Use Random Forest because you'll try multiple model types

```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

def select_features_cv(X, y, n_splits=5, threshold=0.4):
    """
    Select features using Random Forest in CV folds.
    """
    tscv = TimeSeriesSplit(n_splits=n_splits, gap=3)
    feature_counts = Counter()

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]

        # Simple, fast Random Forest for selection
        rf = RandomForestRegressor(
            n_estimators=100,      # Enough for stable importance
            max_depth=10,          # Prevent overfitting
            min_samples_leaf=5,
            random_state=42,
            n_jobs=-1
        )
        rf.fit(X_train, y_train)

        # Keep top 50% by importance
        threshold_imp = np.percentile(rf.feature_importances_, 50)
        selected = X_train.columns[rf.feature_importances_ > threshold_imp]

        feature_counts.update(selected)

    # Return features appearing in ≥40% of folds
    min_appearances = int(threshold * n_splits)  # 2/5 folds
    robust_features = [
        feat for feat, count in feature_counts.items()
        if count >= min_appearances
    ]

    return robust_features

# Use it:
selected_features = select_features_cv(X_train, y_train)

print(f"Selected {len(selected_features)} features from {len(X_train.columns)}")
```

Then:

## Stage 2: Model Comparison
Try different models with the SAME selected features

```python
X_train_sel = X_train[selected_features]
X_test_sel = X_test[selected_features]

models_to_compare = {
    'xgboost': XGBRegressor(n_estimators=1000, learning_rate=0.01),
    'catboost': CatBoostRegressor(iterations=1000, learning_rate=0.05),
    'random_forest': RandomForestRegressor(n_estimators=500),
    # Prophet and SARIMA handle features differently, see below
}

results = {}
for name, model in models_to_compare.items():
    model.fit(X_train_sel, y_train)
    preds = model.predict(X_test_sel)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    results[name] = rmse

    # Log to MLflow
    with mlflow.start_run(run_name=f"{name}_h3"):
        mlflow.log_param("model_type", name)
        mlflow.log_param("n_features", len(selected_features))
        mlflow.log_metric("rmse", rmse)
```

### Special Case: Time-Series Models (Prophet, SARIMA)

Important: Prophet and SARIMA use exogenous features differently:

#### For Prophet:
Only use features that make sense as regressors

```python
prophet_features = [f for f in selected_features
                    if 'lag' not in f]  # Prophet handles lags internally

model = Prophet()
for feat in prophet_features:
    model.add_regressor(feat)

# Prepare data in Prophet format
prophet_df = pd.DataFrame({
    'ds': train_dates,
    'y': y_train,
    **{feat: X_train_sel[feat] for feat in prophet_features}
})
model.fit(prophet_df)
```

#### For SARIMA:
Use selected features as exogenous variables

```python
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(
    y_train,
    exog=X_train_sel[selected_features],  # Use all selected features
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1, 12)
)
results = model.fit()
```

Alternative: Model-Specific Selection

If you want to be more rigorous, you could select features separately for each model type:

- Option 1: One feature set for all models (simpler)
features_all = select_features_cv(X, y, selector_model=RandomForestRegressor())

- Option 2: Model-specific feature sets (more complex)
features_for_xgboost = select_features_cv(X, y, selector_model=XGBRegressor())
features_for_linear = select_features_cv(X, y, selector_model=LassoCV())
features_for_ts = select_features_correlation(X, y)  # Correlation-based

Then train each model with its own features

Recommendation: Start with Option 1 (single feature set using Random Forest). Only use Option 2 if you find significant performance
differences.

Summary:

- Feature selection model = Simple, fast model to identify important features (Random Forest)
- Final prediction models = The actual models you'll compare (XGBoost, CatBoost, Prophet, SARIMA, etc.)

Think of it as:
- Feature selection = "Which ingredients are important?" (use a simple recipe)
- Final modeling = "What's the best way to cook those ingredients?" (try multiple recipes)

The feature selection step just reduces dimensionality and removes noise. The final modeling step is where you actually optimize for
prediction performance.
